In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from datetime import timedelta
import numpy as np
import cv2
import os
import shutil

# Run flags

In [ ]:
# -1 Nothing
# 0 Logs
# 1 Rectangle boxes
# 2 Show classified faces
DEBUG = -1
# Choose cosine similarity and fast method
FAST = True
# Delete Output/ folder files at the end
DELETE = False
# Stop video classifier at a certain frame
# > 0 
STOP_FRAME = 2000

In [ ]:
face_cascade_alt_tree   = cv2.CascadeClassifier('Models/haarcascade/haarcascade_frontalface_alt_tree.xml')
face_cascade_alt        = cv2.CascadeClassifier('Models/haarcascade/haarcascade_frontalface_alt.xml')
face_cascade_alt2       = cv2.CascadeClassifier('Models/haarcascade/haarcascade_frontalface_alt2.xml')
face_cascade_default    = cv2.CascadeClassifier('Models/haarcascade/haarcascade_frontalface_default.xml')

In [ ]:
video = cv2.VideoCapture('Dataset/Youtube/train.mp4')

In [ ]:
frame_interval = 5

fps = video.get(cv2.CAP_PROP_FPS)

frames_to_skip = int(frame_interval * fps)

frame_counter = 0

saved_faces_counter = 0

# Faster one classifier

In [ ]:
if FAST == True:
    while video.isOpened:
        ret, frame = video.read()

        if not ret:
            break

        if STOP_FRAME > 0:
            if frame_counter >= STOP_FRAME:
                break

        frame_counter += 1

        if frame_counter % frames_to_skip != 0:
            continue

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces_alt       = face_cascade_alt.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=6)
        if DEBUG == 0:
            print('Alt      - Number of detected faces:', len(faces_alt))

        for i, (x, y, w, h) in enumerate(faces_alt):
            face = frame[y:y+h, x:x+w]
            cv2.imwrite(f'Output/face-{saved_faces_counter}.jpg', face)
            if DEBUG == 1:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            if DEBUG == 2:
                cv2.imshow(f"Cropped Face {i}", face) 
            saved_faces_counter += 1

        cv2.imshow('Frame', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        if DEBUG == 0:
            current_timestamp = timedelta(seconds=int(frame_counter / fps))
            print("Current Timestamp:", current_timestamp)

        next_frame = int(frame_counter + frames_to_skip)
        video.set(cv2.CAP_PROP_POS_FRAMES, next_frame)

# Slower and accurate 3 classifiers

In [ ]:
if FAST == False:
    while video.isOpened:
        ret, frame = video.read()

        if not ret:
            break

        if STOP_FRAME > 0:
            if frame_counter >= STOP_FRAME:
                break

        frame_counter += 1

        if frame_counter % frames_to_skip != 0:
            continue

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces_alt       = face_cascade_alt.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=6)
        faces_alt2      = face_cascade_alt2.detectMultiScale(gray, scaleFactor=1.4, minNeighbors=4)
        faces_default   = face_cascade_default.detectMultiScale(gray, scaleFactor=1.4, minNeighbors=6)  

        if DEBUG == 0:
            print('Alt      - Number of detected faces:', len(faces_alt))
            print('Alt2     - Number of detected faces:', len(faces_alt2))
            print('Default  - Number of detected faces:', len(faces_default))

        if DEBUG == 1:
            for (x, y, w, h) in faces_alt:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            
            for (x, y, w, h) in faces_alt2:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 255, 0), 2)

            for (x, y, w, h) in faces_default:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 255), 2)

        if len(faces_alt) > 0 and len(faces_alt2) > 0 and len(faces_default) > 0:
            for face1_index, (x1, y1, w1, h1) in enumerate(faces_alt):
                face1 = gray[y1:y1+h1, x1:x1+w1]
                face1_resized = cv2.resize(face1, (64, 64))
                face1_flattened = face1_resized.flatten()
                face1_reshaped = face1_flattened.reshape(1, -1)

                for face2_index, (x2, y2, w2, h2) in enumerate(faces_alt2):
                    face2 = gray[y2:y2+h2, x2:x2+w2]
                    face2_resized = cv2.resize(face2, (64, 64))
                    face2_flattened = face2_resized.flatten()
                    face2_reshaped = face2_flattened.reshape(1, -1)

                    for face3_index, (x3, y3, w3, h3) in enumerate(faces_default):
                        face3 = gray[y3:y3+h3, x3:x3+w3]
                        face3_resized = cv2.resize(face3, (64, 64))
                        face3_flattened = face3_resized.flatten()
                        face3_reshaped = face3_flattened.reshape(1, -1)

                        similarity12 = cosine_similarity(face1_reshaped, face2_reshaped)[0, 0]
                        similarity13 = cosine_similarity(face1_reshaped, face3_reshaped)[0, 0]
                        similarity23 = cosine_similarity(face2_reshaped, face3_reshaped)[0, 0]

                        if similarity12 > 0.9 and similarity13 > 0.9 and similarity23 > 0.9:
                            if DEBUG == 0:
                                print(f'''
                                    F1: {str(face1_index)} 
                                    F2: {str(face2_index)}
                                    F3: {str(face3_index)} 
                                    Sim 1, 2: {str(similarity12)}
                                    Sim 1, 3: {str(similarity13)}
                                    Sim 2, 3: {str(similarity23)}
                                ''')

                            _max = max(similarity12, similarity13, similarity23)

                            if similarity12 == _max:
                                x1, y1, w1, h1
                                face = frame[y1:y1+h1, x1:x1+w1]
                            elif similarity13 == _max:
                                x2, y2, w2, h2
                                face = frame[y2:y2+h2, x2:x2+w2]
                            elif similarity23 == _max:
                                face = frame[y3:y3+h3, x3:x3+w3]
                                x3, y3, w3, h3

                            if DEBUG == 2:
                                cv2.imshow(f"Cropped Face {i}", face)
                            cv2.imwrite(f'Output/face-{saved_faces_counter}.jpg', face)
                            saved_faces_counter += 1

        cv2.imshow('Frame', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        if DEBUG == 0:
            current_timestamp = timedelta(seconds=int(frame_counter / fps))
            print("Current Timestamp:", current_timestamp)

        next_frame = int(frame_counter + frames_to_skip)
        video.set(cv2.CAP_PROP_POS_FRAMES, next_frame)

In [ ]:
video.release()

cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
spl_model = tf.keras.models.load_model('Models/vgg16/finetuning_vgg16_cnn_100_epochs.h5')

In [60]:
image = cv2.imread('Output/face-2.jpg')

if DEBUG == 0:
    image.shape
    plt.imshow(image)
    plt.show()


IMAGE_SIZE = (150, 150)
image = cv2.resize(image, IMAGE_SIZE)

if DEBUG == 0:
    image.shape
    plt.imshow(image)
    plt.show()

image_array = np.array(image)/255.0
image_array = image_array[np.newaxis, ...]

if DEBUG == 0:
    image_array.shape

result = spl_model.predict(image_array)

if DEBUG == 0:
    result.shape

# result

predicted_label_index = np.argmax(result)
# predicted_label_index

1/1 [==============================] - 0s 83ms/step


array([[7.6182890e-03, 1.6386960e-02, 1.0417962e-06, 1.3972180e-02,
        2.1472368e-07, 9.6202129e-01]], dtype=float32)

# Helpers

In [ ]:
if DELETE == True:
    folder = 'Output'
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f'Failed to delete {str(file_path)}. Reason: {str(e)}')